### Preferences model

In [1]:
import pysmile
import pysmile_license
import numpy as np
import pandas as pd

from network_functions import calculate_network_utilities, new_screening_strategy, old_screening_strategy, create_folders_logger
from simulations import plot_classification_results
from plots import plot_estimations_w_error_bars, plot_screening_counts

from preprocessing import preprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt

import yaml
with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

import argparse
import pdb
import matplotlib
matplotlib.use('Agg')

import logging
import datetime 
import os

In [2]:
# from main import main
from use_case_new_strategy import use_case_new_strategy
from full_example import full_example
from update import update_influence_diagram
from network_functions import create_folders_logger
from sens_analysis_elicitation import sens_analysis_elicitation
from sens_analysis_PE_method import sens_analysis_PE_method

In [3]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

## Use cases

### Individual cases

**Benchmark patient**

In [9]:
label = 'notebook_benchmark_patient'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2025-01-14 10:43:25,207 - INFO - Model type: linear
2025-01-14 10:43:25,209 - INFO - Reading network...
2025-01-14 10:43:25,260 - INFO - Calculating information values...
2025-01-14 10:45:28,595 - INFO - Eliciting value of comfort...
2025-01-14 10:45:28,597 - INFO - No elicitation of lambda values, taking default values...
2025-01-14 10:45:28,599 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-14 10:45:28,601 - INFO - Saving network...
2025-01-14 10:45:28,692 - INFO - Plotting info functions...
2025-01-14 10:46:51,607 - INFO - Calculating final utilities...
2025-01-14 10:46:51,629 - INFO - Best info: 15.299634978175163
2025-01-14 10:46:51,631 - INFO - Worst info: 4.450147717014403e-308
2025-01-14 10:46:51,634 - INFO - PE info: 4
2025-01-14 10:46:51,636 - INFO - v_best: 104.06788025218766
2025-01-14 10:46:51,638 - INFO - v_worst: -3.91023528629925

**Benchmark patient risk seeking**

In [10]:
label = 'notebook_benchmark_risk_seeking'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    change_risk_param=True,
    rho_param= 0.005
)

2025-01-14 10:46:52,235 - INFO - Model type: linear
2025-01-14 10:46:52,235 - INFO - Model type: linear
2025-01-14 10:46:52,238 - INFO - Reading network...
2025-01-14 10:46:52,238 - INFO - Reading network...
2025-01-14 10:46:52,281 - INFO - Calculating information values...
2025-01-14 10:46:52,281 - INFO - Calculating information values...
2025-01-14 10:48:52,419 - INFO - Eliciting value of comfort...
2025-01-14 10:48:52,419 - INFO - Eliciting value of comfort...
2025-01-14 10:48:52,424 - INFO - No elicitation of lambda values, taking default values...
2025-01-14 10:48:52,424 - INFO - No elicitation of lambda values, taking default values...
2025-01-14 10:48:52,427 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-14 10:48:52,427 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.8019

**Benchmark patient added evidence**

In [11]:
label = 'notebook_benchmark_addded_ev'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = cfg["patient_chars"].copy()
patient_chars["Diabetes"] = True
patient_chars["Hypertension"] = True

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
)

2025-01-14 10:49:32,446 - INFO - Model type: linear
2025-01-14 10:49:32,447 - INFO - Reading network...
2025-01-14 10:49:32,470 - INFO - Calculating information values...
2025-01-14 10:50:22,158 - INFO - Eliciting value of comfort...
2025-01-14 10:50:22,160 - INFO - No elicitation of lambda values, taking default values...
2025-01-14 10:50:22,160 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-14 10:50:22,161 - INFO - Saving network...
2025-01-14 10:50:22,199 - INFO - Plotting info functions...
2025-01-14 10:50:53,771 - INFO - Calculating final utilities...
2025-01-14 10:50:53,779 - INFO - Best info: 15.299634978175163
2025-01-14 10:50:53,779 - INFO - Worst info: 4.450147717014403e-308
2025-01-14 10:50:53,780 - INFO - PE info: 4
2025-01-14 10:50:53,781 - INFO - v_best: 104.06788025218766
2025-01-14 10:50:53,781 - INFO - v_worst: -3.91023528629925

**Exogenous variable p = 0.02**

In [12]:
label = 'notebook_exogenous_var_002'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }


update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.02
)

2025-01-14 10:50:53,914 - INFO - Model type: linear
2025-01-14 10:50:53,915 - INFO - Reading network...
2025-01-14 10:50:53,930 - INFO - Calculating information values...
2025-01-14 10:51:44,349 - INFO - Eliciting value of comfort...
2025-01-14 10:51:44,350 - INFO - No elicitation of lambda values, taking default values...
2025-01-14 10:51:44,351 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-14 10:51:44,352 - INFO - Saving network...
2025-01-14 10:51:44,394 - INFO - Plotting info functions...
2025-01-14 10:52:17,425 - INFO - Calculating final utilities...
2025-01-14 10:52:17,431 - INFO - Best info: 15.299634978175163
2025-01-14 10:52:17,431 - INFO - Worst info: 4.450147717014403e-308
2025-01-14 10:52:17,432 - INFO - PE info: 4
2025-01-14 10:52:17,433 - INFO - v_best: 104.06788025218766
2025-01-14 10:52:17,433 - INFO - v_worst: -3.91023528629925

**Exogenous variable p = 0.1**

In [13]:
label = 'notebook_exogenous_var_01'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.1
)

2025-01-14 10:52:17,558 - INFO - Model type: linear
2025-01-14 10:52:17,560 - INFO - Reading network...
2025-01-14 10:52:17,578 - INFO - Calculating information values...
2025-01-14 10:53:08,073 - INFO - Eliciting value of comfort...
2025-01-14 10:53:08,074 - INFO - No elicitation of lambda values, taking default values...
2025-01-14 10:53:08,075 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-14 10:53:08,075 - INFO - Saving network...
2025-01-14 10:53:08,118 - INFO - Plotting info functions...
2025-01-14 10:53:38,954 - INFO - Calculating final utilities...
2025-01-14 10:53:38,961 - INFO - Best info: 15.299634978175163
2025-01-14 10:53:38,961 - INFO - Worst info: 4.450147717014403e-308
2025-01-14 10:53:38,962 - INFO - PE info: 4
2025-01-14 10:53:38,963 - INFO - v_best: 104.06788025218766
2025-01-14 10:53:38,963 - INFO - v_worst: -3.91023528629925

### Assessing the French strategy

**Patient 1**

In [14]:
label = 'notebook_4_2_patient_1'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_5_old_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": False,
    "Hypertension": False,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2025-01-14 10:53:39,114 - INFO - Model type: linear
2025-01-14 10:53:39,115 - INFO - Reading network...
2025-01-14 10:53:39,129 - INFO - Calculating information values...
2025-01-14 10:54:30,573 - INFO - Eliciting value of comfort...
2025-01-14 10:54:30,575 - INFO - No elicitation of lambda values, taking default values...
2025-01-14 10:54:30,577 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-14 10:54:30,577 - INFO - Saving network...
2025-01-14 10:54:30,639 - INFO - Plotting info functions...
2025-01-14 10:55:03,576 - INFO - Calculating final utilities...
2025-01-14 10:55:03,583 - INFO - Best info: 15.299634978175163
2025-01-14 10:55:03,583 - INFO - Worst info: 4.450147717014403e-308
2025-01-14 10:55:03,584 - INFO - PE info: 4
2025-01-14 10:55:03,584 - INFO - v_best: 104.06788025218766
2025-01-14 10:55:03,586 - INFO - v_worst: -3.91023528629925

**Patient 2**

In [15]:
label = 'notebook_4_2_patient_2'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": True,
    "Hypertension": True,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2025-01-14 10:55:03,701 - INFO - Model type: linear
2025-01-14 10:55:03,701 - INFO - Reading network...
2025-01-14 10:55:03,716 - INFO - Calculating information values...
2025-01-14 10:55:53,718 - INFO - Eliciting value of comfort...
2025-01-14 10:55:53,719 - INFO - No elicitation of lambda values, taking default values...
2025-01-14 10:55:53,720 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-14 10:55:53,721 - INFO - Saving network...
2025-01-14 10:55:53,759 - INFO - Plotting info functions...
2025-01-14 10:56:26,154 - INFO - Calculating final utilities...
2025-01-14 10:56:26,161 - INFO - Best info: 15.299634978175163
2025-01-14 10:56:26,163 - INFO - Worst info: 4.450147717014403e-308
2025-01-14 10:56:26,163 - INFO - PE info: 4
2025-01-14 10:56:26,164 - INFO - v_best: 104.06788025218766
2025-01-14 10:56:26,165 - INFO - v_worst: -3.91023528629925

**Patient 3**

In [16]:
label = 'notebook_4_2_patient_3'
logger, log_dir = create_folders_logger(label = label)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_1",
    "Smoking": "sm_3_ex_smoker",
    "BMI": "bmi_3_overweight",
    "Alcohol": "high",
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2025-01-14 10:56:26,411 - INFO - Model type: linear
2025-01-14 10:56:26,412 - INFO - Reading network...
2025-01-14 10:56:26,433 - INFO - Calculating information values...
2025-01-14 10:57:17,924 - INFO - Eliciting value of comfort...
2025-01-14 10:57:17,925 - INFO - No elicitation of lambda values, taking default values...
2025-01-14 10:57:17,926 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-14 10:57:17,927 - INFO - Saving network...
2025-01-14 10:57:17,981 - INFO - Plotting info functions...
2025-01-14 10:57:48,624 - INFO - Calculating final utilities...
2025-01-14 10:57:48,632 - INFO - Best info: 15.299634978175163
2025-01-14 10:57:48,632 - INFO - Worst info: 4.450147717014403e-308
2025-01-14 10:57:48,633 - INFO - PE info: 4
2025-01-14 10:57:48,634 - INFO - v_best: 104.06788025218766
2025-01-14 10:57:48,634 - INFO - v_worst: -3.91023528629925

### Designing a national personalised strategy

In [17]:
label = 'notebook_personalised_screening_str'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    logger = logger,
    single_run=True,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = ''
)

2025-01-14 10:57:48,753 - INFO - Configuration variables of interest:
2025-01-14 10:57:48,754 - INFO - Single run: True
2025-01-14 10:57:48,754 - INFO - Use all variables: True
2025-01-14 10:57:48,755 - INFO - Use case with new test: False
2025-01-14 10:57:48,756 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2025-01-14 10:57:48,757 - INFO - Reading the network file...
2025-01-14 10:57:48,772 - INFO - Located at: outputs/linear_rel_point_cond_mut_info_elicitFalse_newtestFalse/decision_models/DM_screening_rel_point_cond_mut_info_linear.xdsl
2025-01-14 10:57:48,774 - INFO - Comfort values: 1 - 4.011942483017116, 2 - 4.168480067521723, 3 - 6.801984517973132, 4 - 7.0
2025-01-14 10:57:49,367 - INFO - All variables are kept in the dataframe for calculation of utilities.
2025-01-14 10:57:49,368 - INFO - Operational limits for the screening strategies: {'No_scr_no_col': inf, 'No_scr_col': 3000, 'gFOBT': 30000, 'FIT': 42000, 'Blood_based': 7000, 'Stool_DNA': 6000, 'CTC': 2000,

### Benchmarking of new screening devices

In [18]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

In [19]:
label = 'notebook_benchmark_new_screening'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = True,
    logger = logger,
    output_dir = 'notebook/' + label
)

2025-01-14 10:59:32,164 - INFO - Model type: linear
2025-01-14 10:59:32,165 - INFO - Reading network...
2025-01-14 10:59:32,187 - INFO - Calculating information values...
2025-01-14 11:00:19,933 - INFO - Eliciting value of comfort...
2025-01-14 11:00:19,934 - INFO - No elicitation of lambda values, taking default values...
2025-01-14 11:00:19,935 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-14 11:00:19,936 - INFO - Adding new test values...
2025-01-14 11:01:22,383 - INFO - Saving network...
2025-01-14 11:01:22,445 - INFO - Plotting info functions...
2025-01-14 11:02:35,128 - INFO - Calculating final utilities...
2025-01-14 11:02:35,136 - INFO - Best info: 15.299634978175163
2025-01-14 11:02:35,137 - INFO - Worst info: 4.450147717014403e-308
2025-01-14 11:02:35,138 - INFO - PE info: 4
2025-01-14 11:02:35,138 - INFO - v_best: 104.06788025218766


In [20]:
cfg["lambda_list"]

[4.011942483017116, 4.168480067521723, 6.801984517973132, 7.0]

In [21]:
label = 'notebook_new_test_screening_strategy'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    use_case_new_test = True,
    single_run=True,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = '',
)

2025-01-14 11:02:35,320 - INFO - Configuration variables of interest:
2025-01-14 11:02:35,320 - INFO - Configuration variables of interest:
2025-01-14 11:02:35,322 - INFO - Single run: True
2025-01-14 11:02:35,322 - INFO - Single run: True
2025-01-14 11:02:35,324 - INFO - Use all variables: True
2025-01-14 11:02:35,324 - INFO - Use all variables: True
2025-01-14 11:02:35,325 - INFO - Use case with new test: True
2025-01-14 11:02:35,325 - INFO - Use case with new test: True
2025-01-14 11:02:35,326 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2025-01-14 11:02:35,326 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2025-01-14 11:02:35,327 - INFO - Reading the network file...
2025-01-14 11:02:35,327 - INFO - Reading the network file...
2025-01-14 11:02:35,345 - INFO - Located at: outputs/linear_rel_point_cond_mut_info_elicitFalse_newtestTrue/decision_models/DM_screening_rel_point_cond_mut_info_linear_new_test.xdsl
2025-01-14 11:02:35,345 - INFO - Located

## Sensitivity analysis

### Sensitivity analysis for the PE

In [4]:
label = 'notebook_sens_analysis_PE_method'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

sens_analysis_PE_method(
    label = label,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,  
)

2025-01-23 14:26:12,796 - INFO - PE_info_array: [4.2 4.  3.8]
2025-01-23 14:26:12,796 - INFO - PE_info_array: [4.2 4.  3.8]
2025-01-23 14:26:12,798 - INFO - PE_cost_array: [ 5 10 50]
2025-01-23 14:26:12,798 - INFO - PE_cost_array: [ 5 10 50]
2025-01-23 14:26:16,539 - INFO - Params: [1.016419338142636, 0.8801528395772084, 0.03681253893587941]
2025-01-23 14:26:16,539 - INFO - Params: [1.016419338142636, 0.8801528395772084, 0.03681253893587941]
2025-01-23 14:27:18,829 - INFO - PE_info: 4.2, PE_cost: 5, DONE!
2025-01-23 14:27:18,829 - INFO - PE_info: 4.2, PE_cost: 5, DONE!
2025-01-23 14:27:19,740 - INFO - Params: [1.0260096777692866, 0.889649200484113, 0.036469730286183666]
2025-01-23 14:27:19,740 - INFO - Params: [1.0260096777692866, 0.889649200484113, 0.036469730286183666]
2025-01-23 14:28:23,001 - INFO - PE_info: 4.2, PE_cost: 10, DONE!
2025-01-23 14:28:23,001 - INFO - PE_info: 4.2, PE_cost: 10, DONE!
2025-01-23 14:28:23,652 - INFO - Params: [1.0235627479785547, 0.8842131758314179, 0.03

### Sensitivity analysis for comfort parameters

In [12]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

sens_analysis_elicitation(
    label = label,
    output_dir = 'notebook/' + label
)

2024-12-30 14:28:45,167 - INFO - Configuration variables of interest:
2024-12-30 14:28:45,167 - INFO - Configuration variables of interest:
2024-12-30 14:28:45,167 - INFO - Configuration variables of interest:
2024-12-30 14:28:45,167 - INFO - Configuration variables of interest:
2024-12-30 14:28:45,170 - INFO - Single run: True
2024-12-30 14:28:45,170 - INFO - Single run: True
2024-12-30 14:28:45,170 - INFO - Single run: True
2024-12-30 14:28:45,170 - INFO - Single run: True
2024-12-30 14:28:45,171 - INFO - Use all variables: True
2024-12-30 14:28:45,171 - INFO - Use all variables: True
2024-12-30 14:28:45,171 - INFO - Use all variables: True
2024-12-30 14:28:45,171 - INFO - Use all variables: True
2024-12-30 14:28:45,173 - INFO - Use case with new test: False
2024-12-30 14:28:45,173 - INFO - Use case with new test: False
2024-12-30 14:28:45,173 - INFO - Use case with new test: False
2024-12-30 14:28:45,173 - INFO - Use case with new test: False
2024-12-30 14:28:45,174 - INFO - PE meth

Counts: best_option
No_scr_no_col    287920
No_scr_col            0
gFOBT                 0
FIT                4131
Blood_based           0
Stool_DNA         47656
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [13]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"].copy()
lambda_list[2] = 6.3

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2024-12-30 14:31:04,567 - INFO - Configuration variables of interest:
2024-12-30 14:31:04,567 - INFO - Configuration variables of interest:
2024-12-30 14:31:04,569 - INFO - Single run: True
2024-12-30 14:31:04,569 - INFO - Single run: True
2024-12-30 14:31:04,570 - INFO - Use all variables: True
2024-12-30 14:31:04,570 - INFO - Use all variables: True
2024-12-30 14:31:04,571 - INFO - Use case with new test: False
2024-12-30 14:31:04,571 - INFO - Use case with new test: False
2024-12-30 14:31:04,572 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-12-30 14:31:04,572 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-12-30 14:31:04,573 - INFO - Change lambdas? True
2024-12-30 14:31:04,573 - INFO - Change lambdas? True
2024-12-30 14:31:04,575 - INFO - Read lambda list from config? True
2024-12-30 14:31:04,575 - INFO - Read lambda list from config? True
2024-12-30 14:31:04,576 - INFO - Lambdas: [4.011942483017116, 4.168480067521723, 6.3, 7.0]
2024-12

Counts: best_option
No_scr_no_col    313631
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA         26076
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [14]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"]
lambda_list[0] = 4.8
lambda_list[1] = 5

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2024-12-30 14:33:24,973 - INFO - Configuration variables of interest:
2024-12-30 14:33:24,973 - INFO - Configuration variables of interest:
2024-12-30 14:33:24,975 - INFO - Single run: True
2024-12-30 14:33:24,975 - INFO - Single run: True
2024-12-30 14:33:24,976 - INFO - Use all variables: True
2024-12-30 14:33:24,976 - INFO - Use all variables: True
2024-12-30 14:33:24,977 - INFO - Use case with new test: False
2024-12-30 14:33:24,977 - INFO - Use case with new test: False
2024-12-30 14:33:24,978 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-12-30 14:33:24,978 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-12-30 14:33:24,979 - INFO - Change lambdas? True
2024-12-30 14:33:24,979 - INFO - Change lambdas? True
2024-12-30 14:33:24,979 - INFO - Read lambda list from config? True
2024-12-30 14:33:24,979 - INFO - Read lambda list from config? True
2024-12-30 14:33:24,980 - INFO - Lambdas: [4.8, 5, 6.801984517973132, 7.0]
2024-12-30 14:33:24,980

Counts: best_option
No_scr_no_col    179713
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA        159994
CTC                   0
Colon_capsule         0
Name: count, dtype: int64
